<b>Realizar uma análise sobre a oferta/vitrine das opções de produtos que repondam a distintas buscas no site do mercadolibre.com.ar</b>

<h1>Tarefa 1</h1>

<n>Varrer uma lista de mais de 150 itens ids no serviço público:

https://api.mercadolibre.com/sites/MLA/search?q=chromecast&limit=50#json

<n>Nesse caso particular e somente como exemplo, são resultado para a busca "chromecast", porém deverá eleger outros términos para o experimento que permitam enriquecer uma análise em um hipotético dashboard (ex: Google Home, Apple TV, Amazon Fire TV, e outros para poder comparar dispositivos portáteis, ou até mesmo eleger outros e que você tenha interesse em comparar).</n></n>

<h3>Criação do Dataframe</h3>

In [147]:
# Imports
import requests
import pandas as pd
import functools as a
import csv

In [148]:
item_range = 10
items = ['chromecast', 'Google Home', 'Apple TV', 'Amazon Fire TV']

# Função que varre os itens e retorna suas respectivas informações
def get_info(item):
    return [requests.get(f"https://api.mercadolibre.com/sites/MLA/search?q={item}&limit={item_range}&offset={num}").json()
            for num in range(1, item_range+1)]

# result_list contém os dicionários de informações para cada combinação de item e número de página
result_list = a.reduce(lambda result, item: result + get_info(item), items, [])

In [106]:
# Converter a lista de dicionários em DataFrame
data_MercadoLibre = pd.DataFrame(result_list)

<h3>Verificação dos Dados</h3>

In [119]:
# Cabeçalho do Dataframe criado
data_MercadoLibre.head()

,site_id,country_default_time_zone,query,paging,results,sort,available_sorts,filters,available_filters
0,MLA,GMT-03:00,chromecast,"{'total': 383, 'primary_results': 383, 'offset...","[{'id': 'MLA1150728234', 'title': 'Google Chro...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'discount', 'name': 'Descuentos', 'typ..."
1,MLA,GMT-03:00,chromecast,"{'total': 383, 'primary_results': 383, 'offset...","[{'id': 'MLA1394845862', 'title': 'Xiaomi Mi T...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'state', 'name': 'Ubicación', 'type': ..."
2,MLA,GMT-03:00,chromecast,"{'total': 383, 'primary_results': 383, 'offset...","[{'id': 'MLA1346181299', 'title': 'Chromecast ...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'discount', 'name': 'Descuentos', 'typ..."
3,MLA,GMT-03:00,chromecast,"{'total': 383, 'primary_results': 383, 'offset...","[{'id': 'MLA1200056325', 'title': 'Google Chro...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'official_store', 'name': 'Tiendas ofi..."
4,MLA,GMT-03:00,chromecast,"{'total': 383, 'primary_results': 383, 'offset...","[{'id': 'MLA1400348604', 'title': 'Xiaomi Stic...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'state', 'name': 'Ubicación', 'type': ..."


In [111]:
# Exemplo do dado desejado
print(data_MercadoLibre['results'][0][1]['id'])

MLA1394845862


In [114]:
# Tipo dos Dados
data_MercadoLibre.dtypes

site_id                      object
country_default_time_zone    object
query                        object
paging                       object
results                      object
sort                         object
available_sorts              object
filters                      object
available_filters            object
dtype: object

In [123]:
data_MercadoLibre[query]

0     {'total': 383, 'primary_results': 383, 'offset...
1     {'total': 383, 'primary_results': 383, 'offset...
2     {'total': 383, 'primary_results': 383, 'offset...
3     {'total': 383, 'primary_results': 383, 'offset...
4     {'total': 383, 'primary_results': 383, 'offset...
5     {'total': 383, 'primary_results': 383, 'offset...
6     {'total': 383, 'primary_results': 383, 'offset...
7     {'total': 383, 'primary_results': 383, 'offset...
8     {'total': 383, 'primary_results': 383, 'offset...
9     {'total': 383, 'primary_results': 383, 'offset...
10    {'total': 67, 'primary_results': 60, 'offset':...
11    {'total': 67, 'primary_results': 60, 'offset':...
12    {'total': 67, 'primary_results': 60, 'offset':...
13    {'total': 67, 'primary_results': 60, 'offset':...
14    {'total': 67, 'primary_results': 60, 'offset':...
15    {'total': 67, 'primary_results': 60, 'offset':...
16    {'total': 67, 'primary_results': 60, 'offset':...
17    {'total': 67, 'primary_results': 60, 'offs

<h1>Tarefa 2</h1>

<n>Para cada resultado, realizar o correspondente GET por item_id no recurso publico:</n>

https://api.mercadolibre.com/items/{Item_Id}

In [144]:
# Obtém as informações do item com base no Item_id.
def get_item_info(item_id):
    url = f"https://api.mercadolibre.com/items/{item_id}"
    response = requests.get(url)

    if response.status_code == 200:
        item_info = response.json()
        return item_info
    else:
        return None

items_mercado_libre = [get_item_info(result['id']) for result in result_list[0]['results']]

<h1>Tarefa 3</h1>

<n>Escrever os resultados em um arquivo plano delimitado por vírgulas, desnormalizando o JSON obtido no passo anterior, em quantos campos sejam necessários para guardar as variáveis que te interesse modelar.</n>

In [160]:
# Lista com os campos desejados do JSON
campos_desejados = ['campo1', 'campo2', 'campo3']

# Caminho do arquivo de saída CSV
arquivo_saida = '\GitHub\mercado_libre_items.csv'

with open(arquivo_saida, 'w', newline='', encoding='utf-8') as arquivo:
    writer = csv.writer(arquivo)
    
    # Escreve o cabeçalho com os nomes dos campos
    writer.writerow(campos_desejados)
    
    # Itera sobre os itens e escreve as informações nos campos correspondentes
    for item in items_mercado_libre:
        linha = []
        
        # Extrai as informações desejadas do item
        for campo in campos_desejados:
            # Verifica se o campo está presente no item
            if campo in item:
                linha.append(item[campo])
            else:
                linha.append('')
        
        # Escreve a linha no arquivo CSV
        writer.writerow(linha)

'resultados.csv'